# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [26]:
!pip install termcolor

  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /root/.cache/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built termcolor


In [27]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag,ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
import sys
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier

from IPython.display import display
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV

from termcolor import colored, cprint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
engine = create_engine('sqlite:///InsertDatabaseName.db')

from sqlalchemy import inspect
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        print("table_name: %s" % table_name)
     #   for column in inspector.get_columns(table_name, schema=schema):
     #       print("Column: %s" % column)

schema: main
table_name: InsertTableName


In [5]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
#X = 'message'
#Y = []

In [6]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [7]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [9]:
X = df.message.values
y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']].values

### 2. Write a tokenization function to process your text data

In [34]:
def tokenize(text):
    # normalise text by making everything lower case
    text = text.lower()
    
    # remove punctuation
    text = re.sub('[^0-9a-zA-Z]',' ',text)
    
    # split text into words
    words = word_tokenize(text)
    #sent  = sent_tokenize(text)
    
    #remove stopwords from words
    words = [word for word in words if word not in stopwords.words('english')]
    
    # part of speech (optional)
    #pos_tag(words)
    
    # use either stemming or lemming to reduce words to their roots
    # reduce words to their stem
    stemmed = [PorterStemmer().stem(w) for w in words]
    #print(stemmed)
    
    # lemmetize words
    #lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed]
    lemmed = [WordNetLemmatizer().lemmatize(w,pos='v') for w in stemmed]
    
    #print(lemmed)

In [35]:
df['message'][0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [36]:
tokenize(df['message'][0])

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']
['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']


In [6]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return tokens

In [11]:
#DONT RUN THIS

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    print("initial text: \n{}\n".format(text))
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = re.sub(url,"urlplaceholder",text)
    print("\ntext after replacing urls: \n{}\n".format(text))
    
    # tokenize text
    tokens = word_tokenize(text)
    
    print("tokens: \n{}\n".format(tokens))
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    print("lemmatizer: \n{}\n".format(lemmatizer))

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok.lower()).strip()
        clean_tokens.append(clean_tok)

    print('clean_tokens: \n{}\n'.format(clean_tokens))
    return clean_tokens

# X, y = load_data()
# for message in X[:5]:
#     tokens = tokenize(message)
#     print(message)
#     print(tokens, '\n')

In [12]:
#RUN this
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

stuff

In [ ]:
# bag of words

from sklearn.feature_extraction.text import CountVectorizer

# initialize count vectorizer object
vect = CountVectorizer(tokenizer=tokenize)

# get counts of each token (word) in text data
X = vect.fit_transform(corpus)

# convert sparse matrix to numpy array to view
X.toarray()

# view token vocabulary and counts
vect.vocabulary_

In [ ]:
# TfidfTransformer

from sklearn.feature_extraction.text import TfidfTransformer

# initialize tf-idf transformer object
transformer = TfidfTransformer(smooth_idf=False)

# use counts from count vectorizer results to compute tf-idf values
tfidf = transformer.fit_transform(X)

# convert sparse matrix to numpy array to view
tfidf.toarray()

In [ ]:
#TfidfVectorizer
#TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

# initialize tf-idf vectorizer object
vectorizer = TfidfVectorizer()

# compute bag of word counts and tf-idf values
X = vectorizer.fit_transform(corpus)

# convert sparse matrix to numpy array to view
X.toarray()

In [ ]:
#one hot encoding

In [ ]:
 / word embending  
        word2Vec = synonyms/antonims a way of working
        GloVe

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
#without a pipeline

In [10]:
import nltk
nltk.download(['punkt', 'wordnet'])
import sys
import os
from sqlalchemy import create_engine
from IPython.display import display
from sklearn.multioutput import MultiOutputClassifier

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


def load_data(database_filepath):

    engine = create_engine('sqlite:///' + database_filepath)
    table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
    df = pd.read_sql_table(table_name,engine)
    
    X = df['message']
    y = df.iloc[:,4:]
    
    
    category_names = y.columns 
    return X, y, category_names


def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)
    
X, y, labels = load_data('DisasterResponse.db')
X_train, X_test, y_train, y_test = train_test_split(X, y)

vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf =  MultiOutputClassifier(RandomForestClassifier())

# train classifier
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)
clf.fit(X_train_tfidf, y_train)

# predict on test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

# display results
#display_results(y_test, y_pred)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
def load_data(database_filepath):

    engine = create_engine('sqlite:///' + database_filepath)
    table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
    df = pd.read_sql_table(table_name,engine)
    
    X = df['message']
    y = df.iloc[:,4:]
    
    
    category_names = y.columns 
    return X, y, category_names, df

X, y, labels, df = load_data('DisasterResponse.db')

In [9]:
from sklearn.pipeline import Pipeline

url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
y_pred = pipeline.predict(X_test)

from sklearn.metrics import classification_report

for col_x in labels:
   # print(col_x)
   # target_names=df[col_x].unique()
   # print(classification_report(y_test,y_pred,target_names=target_names ))
    for i, col in enumerate(labels):
        precision, recall, fscore, support = precision_recall_fscore_support(y_test[col],
                                                                    y_pred[:, i],
                                                                    average='weighted')

        print('\nReport for ({}) column:'.format(colored(col, 'red', attrs=['bold'])))
        
        metrics_params = [
            {'metric': precision, 'name': 'Precision'},
            {'metric': recall, 'name': 'Recall'},
            {'metric': fscore, 'name': 'F-score'}
        ]
        for pmetric in metrics_params:
            if pmetric['metric'] >= 0.75:
                print('  - {}: {}'.format(pmetric['name'],colored(round(pmetric['metric'], 2), 'green')))
            else:
                print('  - {}: {}'.format(pmetric['name'],colored(round(pmetric['metric'], 2), 'yellow')))
    


Report for (related) column:
  - Precision: 0.79
  - Recall: 0.8
  - F-score: 0.79

Report for (request) column:
  - Precision: 0.88
  - Recall: 0.89
  - F-score: 0.87

Report for (offer) column:
  - Precision: 0.99
  - Recall: 1.0
  - F-score: 0.99

Report for (aid_related) column:
  - Precision: 0.73
  - Recall: 0.73
  - F-score: 0.72

Report for (medical_help) column:
  - Precision: 0.9
  - Recall: 0.92
  - F-score: 0.89

Report for (medical_products) column:
  - Precision: 0.94
  - Recall: 0.95
  - F-score: 0.93

Report for (search_and_rescue) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.96

Report for (security) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (military) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.95

Report for (child_alone) column:
  - Precision: 1.0
  - Recall: 1.0
  - F-score: 1.0

Report for (water) column:
  - Precision: 0.94
  - Recall: 0.94
  - F-score: 0.93

Report for (food) column:
  - Precision:

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



  - Recall: 0.96
  - F-score: 0.94

Report for (buildings) column:
  - Precision: 0.95
  - Recall: 0.95
  - F-score: 0.93

Report for (electricity) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (tools) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.99

Report for (hospitals) column:
  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (shops) column:
  - Precision: 0.99
  - Recall: 1.0
  - F-score: 0.99

Report for (aid_centers) column:
  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (other_infrastructure) column:
  - Precision: 0.93
  - Recall: 0.95
  - F-score: 0.93

Report for (weather_related) column:
  - Precision: 0.84
  - Recall: 0.84
  - F-score: 0.83

Report for (floods) column:
  - Precision: 0.93
  - Recall: 0.93
  - F-score: 0.92

Report for (storm) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.92

Report for (fire) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report


Report for (offer) column:
  - Precision: 0.99
  - Recall: 1.0
  - F-score: 0.99

Report for (aid_related) column:
  - Precision: 0.73
  - Recall: 0.73
  - F-score: 0.72

Report for (medical_help) column:
  - Precision: 0.9
  - Recall: 0.92
  - F-score: 0.89

Report for (medical_products) column:
  - Precision: 0.94
  - Recall: 0.95
  - F-score: 0.93

Report for (search_and_rescue) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.96

Report for (security) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (military) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.95

Report for (child_alone) column:
  - Precision: 1.0
  - Recall: 1.0
  - F-score: 1.0

Report for (water) column:
  - Precision: 0.94
  - Recall: 0.94
  - F-score: 0.93

Report for (food) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.91

Report for (shelter) column:
  - Precision: 0.92
  - Recall: 0.92
  - F-score: 0.9

Report for (clothing) column:
  - Precision


Report for (electricity) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (tools) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.99

Report for (hospitals) column:
  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (shops) column:
  - Precision: 0.99
  - Recall: 1.0
  - F-score: 0.99

Report for (aid_centers) column:
  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (other_infrastructure) column:
  - Precision: 0.93
  - Recall: 0.95
  - F-score: 0.93

Report for (weather_related) column:
  - Precision: 0.84
  - Recall: 0.84
  - F-score: 0.83

Report for (floods) column:
  - Precision: 0.93
  - Recall: 0.93
  - F-score: 0.92

Report for (storm) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.92

Report for (fire) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report for (earthquake) column:
  - Precision: 0.96
  - Recall: 0.96
  - F-score: 0.96

Report for (cold) column:
  - Precision: 

...

  - Recall: 0.95
  - F-score: 0.93

Report for (weather_related) column:
  - Precision: 0.84
  - Recall: 0.84
  - F-score: 0.83

Report for (floods) column:
  - Precision: 0.93
  - Recall: 0.93
  - F-score: 0.92

Report for (storm) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.92

Report for (fire) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report for (earthquake) column:
  - Precision: 0.96
  - Recall: 0.96
  - F-score: 0.96

Report for (cold) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (other_weather) column:
  - Precision: 0.93
  - Recall: 0.94
  - F-score: 0.92

Report for (direct_report) column:
  - Precision: 0.85
  - Recall: 0.86
  - F-score: 0.83

Report for (related) column:
  - Precision: 0.79
  - Recall: 0.8
  - F-score: 0.79

Report for (request) column:
  - Precision: 0.88
  - Recall: 0.89
  - F-score: 0.87

Report for (offer) column:
  - Precision: 0.99
  - Recall: 1.0
  - F-score: 0.99

Report for (aid_rel

  - F-score: 0.95

Report for (child_alone) column:
  - Precision: 1.0
  - Recall: 1.0
  - F-score: 1.0

Report for (water) column:
  - Precision: 0.94
  - Recall: 0.94
  - F-score: 0.93

Report for (food) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.91

Report for (shelter) column:
  - Precision: 0.92
  - Recall: 0.92
  - F-score: 0.9

Report for (clothing) column:
  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (money) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (missing_people) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report for (refugees) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.95

Report for (death) column:
  - Precision: 0.95
  - Recall: 0.96
  - F-score: 0.94

Report for (other_aid) column:
  - Precision: 0.82
  - Recall: 0.87
  - F-score: 0.82

Report for (infrastructure_related) column:
  - Precision: 0.89
  - Recall: 0.93
  - F-score: 0.9

Report for (transport) column:

Report for (other_infrastructure) column:
  - Precision: 0.93
  - Recall: 0.95
  - F-score: 0.93

Report for (weather_related) column:
  - Precision: 0.84
  - Recall: 0.84
  - F-score: 0.83

Report for (floods) column:
  - Precision: 0.93
  - Recall: 0.93
  - F-score: 0.92

Report for (storm) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.92

Report for (fire) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report for (earthquake) column:
  - Precision: 0.96
  - Recall: 0.96
  - F-score: 0.96

Report for (cold) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (other_weather) column:
  - Precision: 0.93
  - Recall: 0.94
  - F-score: 0.92

Report for (direct_report) column:
  - Precision: 0.85
  - Recall: 0.86
  - F-score: 0.83

Report for (related) column:
  - Precision: 0.79
  - Recall: 0.8
  - F-score: 0.79

Report for (request) column:
  - Precision: 0.88
  - Recall: 0.89
  - F-score: 0.87

Report for (offer) column:
  - Precisio


Report for (military) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.95

Report for (child_alone) column:
  - Precision: 1.0
  - Recall: 1.0
  - F-score: 1.0

Report for (water) column:
  - Precision: 0.94
  - Recall: 0.94
  - F-score: 0.93

Report for (food) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.91

Report for (shelter) column:
  - Precision: 0.92
  - Recall: 0.92
  - F-score: 0.9

Report for (clothing) column:
  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (money) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (missing_people) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report for (refugees) column:
  - Precision: 0.96
  - Recall: 0.97
  - F-score: 0.95

Report for (death) column:
  - Precision: 0.95
  - Recall: 0.96
  - F-score: 0.94

Report for (other_aid) column:
  - Precision: 0.82
  - Recall: 0.87
  - F-score: 0.82

Report for (infrastructure_related) column:
  - Precision: 0.

  - Precision: 0.98
  - Recall: 0.99
  - F-score: 0.98

Report for (other_infrastructure) column:
  - Precision: 0.93
  - Recall: 0.95
  - F-score: 0.93

Report for (weather_related) column:
  - Precision: 0.84
  - Recall: 0.84
  - F-score: 0.83

Report for (floods) column:
  - Precision: 0.93
  - Recall: 0.93
  - F-score: 0.92

Report for (storm) column:
  - Precision: 0.92
  - Recall: 0.93
  - F-score: 0.92

Report for (fire) column:
  - Precision: 0.99
  - Recall: 0.99
  - F-score: 0.98

Report for (earthquake) column:
  - Precision: 0.96
  - Recall: 0.96
  - F-score: 0.96

Report for (cold) column:
  - Precision: 0.97
  - Recall: 0.98
  - F-score: 0.97

Report for (other_weather) column:
  - Precision: 0.93
  - Recall: 0.94
  - F-score: 0.92

Report for (direct_report) column:
  - Precision: 0.85
  - Recall: 0.86
  - F-score: 0.83

Report for (related) column:
  - Precision: 0.79
  - Recall: 0.8
  - F-score: 0.79

Report for (request) column:
  - Precision: 0.88
  - Recall: 0.89
  

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
def save_model(model, model_filepath):
    """
    Save model to a pickle file
    """
    pickle.dump(model, open(model_filepath, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# inpstall termcolor
!pip install termcolor

# import libraries
import re
import sys
import os
from sqlalchemy import create_engine
from IPython.display import display
import pandas as pd
import nltk
nltk.download(['punkt','words','stopwords','averaged_perceptron_tagger','maxent_ne_chunker','wordnet'])
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag,ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from termcolor import colored, cprint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
def load_data(database_filepath):

    """
    function is used to import the clean data messages from the database
    
    INPUT: filepath of database
    OUTUP: it returns the X,y datasets and the column names
    
    """
    engine = create_engine('sqlite:///' + database_filepath)
    table_name = os.path.basename(database_filepath).replace(".db","") + "_table"
    df = pd.read_sql_table(table_name,engine)
    
    X = df['message']
    y = df.iloc[:,4:]
    
    
    category_names = y.columns 
    return X, y, category_names, df

#X, y, labels, df = load_data('DisasterResponse.db')

In [ ]:
# url find expression
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    
    """function to tokenise, lemmitize and clean the message strings
    
    INPUT: takes the text to be processed
    OUTPUT: returns the clean tokens to be further analysed"""
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

#Split data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y)

def build_model():
    """ build pipeline with transformers ans classifier for the model
    
    INPUT: None
    OUTPUT: model pipeline with tunes parameters"""
    
    # create pipeline
    pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),

                ('starting_verb', StartingVerbExtractor())
            ])),

            ('clf', RandomForestClassifier())
        ])
    
    # choose for model parameters
    parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

def evaluate_model(model, X_test, Y_test, category_names):
    
    """function to evaluate the precision, recall and fscore of the multiple X variables
    
       INPUT: model object, X-test dataset, y-test dataset and column names
       OUTPUT: prints the presicion of the y predicted variables"""
    
    # calculate the predicted y - y_pred
    y_pred = pipeline.predict(X_test)
    
    # get -false-positives of predicted y compared to test y and get accuracy

    for col_x in labels:

        for i, col in enumerate(category_names):
            precision, recall, fscore, support = precision_recall_fscore_support(y_test[col],
                                                                        y_pred[:, i],
                                                                        average='weighted')

            print('\nReport for ({}) column:'.format(colored(col, 'red', attrs=['bold'])))

            metrics_params = [
                {'metric': precision, 'name': 'Precision'},
                {'metric': recall, 'name': 'Recall'},
                {'metric': fscore, 'name': 'F-score'}
            ]
            for pmetric in metrics_params:
                if pmetric['metric'] >= 0.75:
                    print('  - {}: {}'.format(pmetric['name'],colored(round(pmetric['metric'], 2), 'green')))
                else:
                    print('  - {}: {}'.format(pmetric['name'],colored(round(pmetric['metric'], 2), 'yellow')))


In [5]:
# import libraries
import re
import sys
import os
import subprocess
from sqlalchemy import create_engine
from IPython.display import display
import pandas as pd
import nltk
nltk.download(['punkt','words','stopwords','averaged_perceptron_tagger','maxent_ne_chunker','wordnet'])
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag,ne_chunk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

# inpstall termcolor
subprocess.check_call([sys.executable, "-m", "pip", "install", 'termcolor'])
print("termcolor installed")
from termcolor import colored, cprint


def load_data(database_filepath):
    """Load clean dataset from database
    
    inputs:
    filepath: string. Filepath for database file containing messages dataset.
       
    outputs:
    df: dataframe. Dataframe containing clean dataset of messages.
    """

    engine = create_engine('sqlite:///' + database_filepath)
    table_name = database_filepath.split('/')[-1]
    table_name = table_name.split('.')[0]
    df = pd.read_sql_table(table_name,engine)
    
    X = df['message']
    y = df.iloc[:,4:]
    
    
    category_names = y.columns 
    return X, y, category_names
    
   # display(df.head())
    #return df
    
    

class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    
    

def tokenize(text):
    
    # in the message text identify the url to replace it. Expression
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # 
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    


def build_model():
    """ build pipeline with transformers ans classifier for the model
    
    INPUT: None
    OUTPUT: model pipeline with tunes parameters"""
    
    # create pipeline
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
   ])
    
    return pipeline
    

#     parameters = {'vect__min_df': [1, 5],
#                 'tfidf__use_idf': [True, False],
#                 'clf__estimator__n_estimators': [10, 25],
#                 'clf__estimator__min_samples_split': [2, 4]}
    
#     cv = GridSearchCV(pipeline, param_grid=parameters)

#     return cv


def evaluate_model(model, X_test, y_test, category_names):
    
    """function to evaluate the precision, recall and fscore of the multiple X variables
    
       INPUT: model object, X-test dataset, y-test dataset and column names
       OUTPUT: prints the presicion of the y predicted variables"""
    
    
    # calculate the predicted y - y_pred
    y_pred = model.predict(X_test)
    
    # get -false-positives of predicted y compared to test y and get accuracy

    
    for i, col in enumerate(category_names):
        precision, recall, fscore, support = precision_recall_fscore_support(y_test[col],
                                                                    y_pred[:, i],
                                                                    average='weighted')

        print('\nReport for ({}) column:'.format(colored(col, 'red', attrs=['bold'])))

        metrics_params = [
            {'metric': precision, 'name': 'Precision'},
            {'metric': recall, 'name': 'Recall'},
            {'metric': fscore, 'name': 'F-score'}
        ]
        for pmetric in metrics_params:
            if pmetric['metric'] >= 0.75:
                print('  - {}: {}'.format(pmetric['name'],colored(round(pmetric['metric'], 2), 'green')))
            else:
                print('  - {}: {}'.format(pmetric['name'],colored(round(pmetric['metric'], 2), 'yellow')))



def save_model(model, model_filepath):
    """
    Save model to a pickle file
    """
    pickle.dump(model, open(model_filepath, 'wb'))
    print("model saved in pickle file")


database_filepath='DisasterResponse.db'
model_filepath='classifier.pkl'
print('Loading data...\n    DATABASE: {}'.format(database_filepath))
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

print('Building model...')
model = build_model()

print('Training model...')
model.fit(X_train, Y_train)

print('Evaluating model...')
evaluate_model(model, X_test, Y_test, category_names)

print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

print('Trained model saved!!!')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
termcolor installed
Loading data...
    DATABASE: DisasterResponse.db
Building model...
Training model...
Evaluating model...

Report for (related) column:
  - Precision: 0.79
  - Recall: 0.81
  - F-score:

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


model saved in pickle file
Trained model saved!!!
